In [77]:
import pandas as pd
import numpy as np
import scipy.stats as sps

In [78]:
data = pd.read_csv('cps99_ps3.csv')
data

,ahe,female,ne,midwest,south,west,race,yrseduc,ba,hsdipl,age
0,10.754048,1,1,0,0,0,1,18,1,1,49
1,10.341881,0,1,0,0,0,1,16,1,1,28
2,18.181818,0,1,0,0,0,1,12,0,1,45
3,20.879122,1,1,0,0,0,1,12,0,1,41
4,17.948717,0,1,0,0,0,1,18,1,1,54
...,...,...,...,...,...,...,...,...,...,...,...
37805,41.208790,0,0,0,0,1,4,19,1,1,51
37806,21.634615,0,0,0,0,1,4,16,1,1,44
37807,8.636364,1,0,0,0,1,4,14,0,1,44
37808,10.544871,0,0,0,0,1,4,13,0,1,33


In [79]:
y = np.log(data['ahe'])
x = np.array([data.yrseduc, data.age, data.female, data.hsdipl]).T
x = np.column_stack((np.ones(x.shape[0]), x))
beta_hat = np.linalg.inv(x.T @ x) @ x.T @ y

print('a)')
print(f"Коэффициент при yrseduc = {beta_hat[1]}, при age = {beta_hat[2]}, при female = {beta_hat[3]}")
print("Можно сделать вывод, что на зарплату значительно влияет пол человека, при этом в среднем женщины зарабатывают меньше мужчин.")

a)
Коэффициент при yrseduc = 0.08142349709381655, при age = 0.007489681891459749, при female = -0.24409048625868998
Можно сделать вывод, что на зарплату значительно влияет пол человека, при этом в среднем женщины зарабатывают меньше мужчин.


In [80]:
residuals = y - x @ beta_hat
rmse = np.sqrt(np.mean(residuals**2))
print("b)")
print(f'Среднеквадратическое отклонение равно = {rmse}.\nЭто значит, что в среднем зарплата отклоняется от среднего значения по выборке на 0.5')

b)
Среднеквадратическое отклонение равно = 0.46894698536288926.
Это значит, что в среднем зарплата отклоняется от среднего значения по выборке на 0.5


In [81]:
sigma2_hat = np.sum(residuals**2)/(x.shape[0] - x.shape[1])
cov_matrix = sigma2_hat * np.linalg.inv(x.T @ x)
std_error = np.sqrt(np.diag(cov_matrix))
alpha = 0.05
left = beta_hat[3] - sps.t.ppf(1-alpha/2, df=x.shape[0] - x.shape[1])*std_error[3]
right = beta_hat[3] + sps.t.ppf(1-alpha/2, df=x.shape[0] - x.shape[1])*std_error[3]
print('c)')
print(f'95%-ый доверительный интервал: [{left}, {right}]')

c)
95%-ый доверительный интервал: [-0.25366065259532944, -0.2345203199220505]


In [82]:
# H0: new_beta_hat[4] (эффект получения диплома) = 0
# Ha: new_beta_hat[4] != 0

effect = beta_hat[4]
t_stat = effect / std_error[4]

p_value = 2 * (1 - sps.t.cdf(np.abs(t_stat), df=x.shape[0] - x.shape[1]))

print('d)')
print(f'p-value = {p_value}')
if p_value > alpha:
    print('Нулевая гипотеза не отвергается. Наличие диплома оказывает значимый эффект на зарплату.')
else:
    print('Нулевая гипотеза отвергается. Наличие диплома не оказывает значимого эффекта на зарплату.')

d)
p-value = 0.0
Нулевая гипотеза отвергается. Наличие диплома не оказывает значимого эффекта на зарплату.


In [83]:
V = (np.array([1, 12, 30, 0, 1]).T) @ beta_hat - (np.array([1, 10, 30, 0, 0]).T) @ beta_hat
print('e)')
print(f"V = {V}")

e)
V = 0.2898395282796029


In [84]:
# так как V = 2*beta_hat[1] + beta_hat[4] => var_V = 4*V(yrseduc_hat) + V(hsdipl_hat) + 4*cov(yrseduc_hat, hsdipl_hat)
var_V = 4*cov_matrix[1][1] + cov_matrix[4][4] + 4*cov_matrix[1][4]
std_err_V = np.sqrt(var_V)

v_left = V - sps.t.ppf(1-alpha/2, df=x.shape[0] - x.shape[1])*std_err_V
v_right = V + sps.t.ppf(1-alpha/2, df=x.shape[0] - x.shape[1])*std_err_V

print('f)')
print(f'95%-ый доверительный интервал для V: [{v_left}, {v_right}]')

f)
95%-ый доверительный интервал для V: [0.27267895922735425, 0.3070000973318516]
